In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [2]:
load_dotenv()

# os.getenv("KEY_NAME")

True

In [36]:
llm = ChatOpenAI(openai_api_key = os.getenv("OPENAI_API_KEY"), model_name = "gpt-3.5-turbo", temperature = 0.7)

In [4]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker.Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} in {tone}. Make sure the questions are not repeated and check all the questions to be conforming.Make sure to format your response like RESPONSE_JSON below and use it as a guide.
Ensure to make {number} MCQs
### Response_JSON
{response_json}
"""

In [5]:
with open("../response.json","r") as f:
    response_json = json.load(f)
    
# print(response_json)

In [6]:
mcq_gen_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [7]:
mcq_gen_chain = LLMChain(llm = llm, prompt = mcq_gen_prompt, output_key = "quiz", verbose = True)

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [8]:
TEMPLATE2 = """
You are an expert english grammerian and writer. Given a Multiple Choice Question for {subject} you need to evaluate the complexity of the question and give a complete analysis. If the quiz is not at per with the cognitive and analytical abilities of the students update the quiz questions which needs to be changed and change the tone such that is more suitable for the students.
Quiz_mcqs
{quiz}

Check from an expert English Writer of the above quiz. 
"""

In [9]:
evaluator_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template = TEMPLATE2
)

In [10]:
evaluator_chain = LLMChain(llm = llm, prompt = evaluator_prompt, output_key = "review", verbose = True)

In [12]:
generate_evaluation_chain = SequentialChain(chains = [mcq_gen_chain, evaluator_chain], input_variables=["text", "number","subject","tone","response_json"], verbose = True, output_variables=["quiz", "review"])

In [18]:
with open("../data.txt","r") as file:
    text = file.read()
    
print(text)

Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.

AI technology is widely used throughout industry, government, and science. Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go).[2] However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications,

In [ ]:
with get_openai_callback() as cb:
    response = generate_evaluation_chain(
        {"text": text,
         "number": 5,
         "subject": "Computer Science",
         "tone": "formal",
         "response_json": json.dumps(response_json)}
)

In [ ]:
quiz = response.get("quiz")

quiz = json.loads(quiz)

# format this key value pair.